<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import numpy as np
import pandas as pd
import os as os

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [2]:
# Create an output folder.
#https://www.tutorialspoint.com/python/os_mkdir.htm
#https://docs.python.org/3/library/os.html
#https://pythonexamples.org/python-create-directory-mkdir/

#os.mkdir('output')
#the folder was created when I ran the above code for the first
#time. I've commented it out to avoid getting a repeat error.

# Create a files variable that contains all of our data files.
#https://www.tutorialspoint.com/python/os_listdir.htm
files = os.listdir('data/')

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [3]:
# Read in data from your city from January 1st.
#https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
#journaldev.com/33306/pandas-read_excel-reading-excel-file-in-python

jan_1 = pd.read_excel('data/Jan 1.xlsx', sheet_name = 'Boston')
jan_1.head()

#https://exerror.com/workbook-contains-no-default-style-apply-openpyxls-default/
#I tried to get rid of the error by the following the instructions
#in the above link but couldn't get it to work. 

C:\Users\Mary\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,prodcode,price_eu,weight_kg,quantity
0,4134,1.957576,7.857868,498
1,4012,0.912565,5.746554,384
2,4323,0.740753,8.748904,431
3,4078,1.571533,7.061995,457
4,4323,1.389404,1.117369,316


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [4]:
jan_1['price_usd'] = jan_1['price_eu'] * 1.1
jan_1.head()

,prodcode,price_eu,weight_kg,quantity,price_usd
0,4134,1.957576,7.857868,498,2.153334
1,4012,0.912565,5.746554,384,1.003821
2,4323,0.740753,8.748904,431,0.814828
3,4078,1.571533,7.061995,457,1.728686
4,4323,1.389404,1.117369,316,1.528344


In [5]:
jan_1['weight_lb'] = jan_1['weight_kg'] * 2.2
jan_1.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4134,1.957576,7.857868,498,2.153334,17.287309
1,4012,0.912565,5.746554,384,1.003821,12.642420
2,4323,0.740753,8.748904,431,0.814828,19.247589
3,4078,1.571533,7.061995,457,1.728686,15.536389
4,4323,1.389404,1.117369,316,1.528344,2.458212


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [6]:
plu = pd.read_csv("plu-codes.csv")
plu.head()

,product,plu_code
0,Apple (Fuji),4131
1,Apple (Gala),4134
2,Apricot,3302
3,Avocado,4225
4,Banana,4011


In [7]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
plu.rename(columns = {'plu_code':  'prodcode'}, inplace = True)

In [8]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
#https://stackoverflow.com/questions/53645882/pandas-merging-101
jan_1 = jan_1.merge(plu, how = 'left')
jan_1.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product
0,4134,1.957576,7.857868,498,2.153334,17.287309,Apple (Gala)
1,4012,0.912565,5.746554,384,1.003821,12.642420,Orange
2,4323,0.740753,8.748904,431,0.814828,19.247589,Strawberries
3,4078,1.571533,7.061995,457,1.728686,15.536389,Corn
4,4323,1.389404,1.117369,316,1.528344,2.458212,Strawberries


### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [9]:
jan_1.drop(['price_eu', 'weight_kg'], axis = 1)

,prodcode,quantity,price_usd,weight_lb,product
0,4134,498,2.153334,17.287309,Apple (Gala)
1,4012,384,1.003821,12.642420,Orange
2,4323,431,0.814828,19.247589,Strawberries
3,4078,457,1.728686,15.536389,Corn
4,4323,316,1.528344,2.458212,Strawberries
...,...,...,...,...,...
172,4131,309,0.864199,7.000650,Apple (Fuji)
173,4958,144,1.946128,3.702045,Lemon
174,4272,455,2.057089,9.947643,Grapes
175,3302,482,0.936777,2.459543,Apricot


### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [10]:
jan_1['date'] = 'Jan 1'

### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [11]:
def process_data(file, city):
    #reading in the file
    df_file = pd.read_excel('data/'+file, sheet_name = city)

    #creating the new columns in FREEDOM UNITS
    df_file['price_usd'] = df_file['price_eu'] * 1.1
    df_file['weight_lb'] = df_file['weight_kg'] * 2.2
    
    #dropping the columns with COMMIE UNITS
    df_file.drop(['price_eu', 'weight_kg'], axis = 1, inplace = True)
    
    #renaming prodcode column to merge with plu .csv
    df_file = df_file.merge(plu, how = 'left')
    
    #creating the column specific to date
    df_file['date'] = file.strip('.xlsx')
    
    return df_file

Test your function out on a new file and city!

In [12]:
process_data('Jan 2.xlsx', 'Boston')

C:\Users\Mary\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,prodcode,quantity,price_usd,weight_lb,product,date
0,4030,109,1.974055,16.298007,Kiwi,Jan 2
1,4796,329,2.712247,10.411764,Tomato,Jan 2
2,4309,202,2.601073,5.135373,Lychee,Jan 2
3,4062,377,2.943422,12.706182,Cucumber,Jan 2
4,4062,181,2.798499,15.870113,Cucumber,Jan 2
...,...,...,...,...,...,...
141,4159,236,0.724818,20.015250,Onion,Jan 2
142,3302,307,1.023866,8.104321,Apricot,Jan 2
143,4240,498,2.855083,12.930640,Blueberries,Jan 2
144,3302,265,0.729489,13.297231,Apricot,Jan 2


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [13]:
boston_data = [process_data(file, 'Boston') for file in files]

C:\Users\Mary\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [14]:
#https://datacarpentry.org/python-ecology-lesson/05-merging-data/
boston_jan = pd.concat(boston_data, axis = 0)
boston_jan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4590 entries, 0 to 116
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   prodcode   4590 non-null   int64  
 1   quantity   4590 non-null   int64  
 2   price_usd  4590 non-null   float64
 3   weight_lb  4590 non-null   float64
 4   product    4590 non-null   object 
 5   date       4590 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 251.0+ KB


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [15]:
def process_data(file, city):
    #reading in the file
    df_file = pd.read_excel('data/' + file, sheet_name = city)
    
    #creating a new column 'city'
    df_file['city'] = city

    #creating the new columns in FREEDOM UNITS
    df_file['price_usd'] = df_file['price_eu'] * 1.1
    df_file['weight_lb'] = df_file['weight_kg'] * 2.2
    
    #dropping the columns with COMMIE UNITS
    df_file.drop(['price_eu', 'weight_kg'], axis = 1, inplace = True)
    
    #renaming prodcode column to merge with plu .csv
    df_file = df_file.merge(plu, how = 'left')
    
    #creating the column specific to date
    df_file['date'] = file.strip('.xlsx')
    
    #re-ordering the columns with .loc
    df_file = df_file.loc[:, ['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]
    
    return df_file

In [16]:
#testing out above function
Atl_sample_df = process_data('Jan 1.xlsx', 'Atlanta')
Atl_sample_df.head()

,city,date,product,prodcode,quantity,weight_lb,price_usd
0,Atlanta,Jan 1,Grapes,4272,155,10.184810,1.285202
1,Atlanta,Jan 1,Peach,4404,325,4.741293,2.196151
2,Atlanta,Jan 1,Apple (Fuji),4131,418,18.439256,2.223649
3,Atlanta,Jan 1,Peach,4404,177,16.391065,3.278436
4,Atlanta,Jan 1,Mushroom,4650,384,17.808832,1.075772


In [17]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [18]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

#https://www.pythontutorial.net/python-basics/python-write-csv-file/

#looping through cities in city_dict
for city in city_dict.keys():
    
    #establishing empty dataframe
    city_df = []
    
    #looping through our files in files
    for file in files:
        
        #calling our process_data function to return dataframe
        city_df.append(process_data(file, city))
        
    #concatenating the different file data frames in files and printing to .csv    
    city_data = pd.concat(city_df, axis = 0)
    city_data.to_csv(f'./output/{city}.csv')

C:\Users\Mary\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [19]:
#reading in the csvs with list comprehension
all_cities = [pd.read_csv(f'output/{city}.csv') for city in city_dict.keys()]

#concating the different cities' data into a dataframe
all_cities_jan = pd.concat(all_cities, axis = 0)

#droping the new index column formed
all_cities_jan.drop('Unnamed: 0', axis = 1, inplace = True)

#displaying first five rows of data frame
all_cities_jan.head()

,city,date,product,prodcode,quantity,weight_lb,price_usd
0,Atlanta,Jan 1,Grapes,4272,155,10.184810,1.285202
1,Atlanta,Jan 1,Peach,4404,325,4.741293,2.196151
2,Atlanta,Jan 1,Apple (Fuji),4131,418,18.439256,2.223649
3,Atlanta,Jan 1,Peach,4404,177,16.391065,3.278436
4,Atlanta,Jan 1,Mushroom,4650,384,17.808832,1.075772


In [20]:
all_cities_jan.drop(['date', 'product', 'prodcode'], axis = 1, inplace = True)
all_cities_jan.head()

,city,quantity,weight_lb,price_usd
0,Atlanta,155,10.184810,1.285202
1,Atlanta,325,4.741293,2.196151
2,Atlanta,418,18.439256,2.223649
3,Atlanta,177,16.391065,3.278436
4,Atlanta,384,17.808832,1.075772


In [21]:
#https://stackoverflow.com/questions/28236305/how-do-i-sum-values-in-a-column-that-match-a-given-condition-using-pandas
all_cities_jan.loc[all_cities_jan['city'] == 'Atlanta', 'quantity'].mean()

#I've got the data correct, now I need to make the DataFrame of mean values. 

302.4853064651553

In [22]:
#https://stackoverflow.com/questions/46826773/how-can-i-merge-rows-by-same-value-in-a-column-in-pandas-with-aggregation-func?rq=1
all_cities_new = all_cities_jan.groupby(all_cities_jan['city']).mean()
all_cities_new

,quantity,weight_lb,price_usd
city,,,
Atlanta,302.485306,11.929027,1.937980
Austin,301.048298,12.092775,1.937456
Boston,298.806536,12.063057,1.900973
Chicago,301.686186,12.101555,1.930026
Denver,301.012745,12.129850,1.932088
Los Angeles,301.531276,12.167527,1.918331
New York,299.482863,12.090294,1.912662
San Francisco,298.979895,12.237399,1.925830
Seattle,300.333694,11.925486,1.903315


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).